# RHESSys model setup script

![alt text](https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/rhessys_filesystem.png "Logo Title Text 1")

https://github.com/laurencelin/GIS2RHESSys

***
## 1) RHESSys project directory & upload files

### 1.A define directory names

<div class="alert alert-block alert-danger">
<b>Cuction:</b> Please reload the blocks of codes below for each session.
These two cells below setup the directory structures.
</div>

In [ ]:
%%bash --out SCRATCH 
printf "/scratch/$USER"

In [ ]:
# -------------------------- project and RHESSys
PROJDIR='jupyter_ws18' #***
RHESSysMODEL='ws18_local' #***
# -------------------------- Other variables and paths (do not modify)
RAWGISDIR=SCRATCH+'/'+PROJDIR+'/raw_data'
RHESSysDIR=SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL
GISDBASE=SCRATCH+'/'+PROJDIR+'/grass_dataset'
RBASE=SCRATCH+'/'+PROJDIR+'/RLIB'
# -------------------------- GIS spatial resolution and projection (UTM)
# look up from http://spatialreference.org/ref/epsg/?page=1
# EPSG:26917 = NAD83 UTM 17N
# EPSG:26918 = NAD83 UTM 18N
EPSGCODE='EPSG:26917' # NAD83 UTM 17N ***
RESOLUTION=10 #***
MAPSET='PERMANENT'
# -----------------------------------------
LOCATION=GISDBASE+'/'+RHESSysMODEL
LOCATIONDEM=GISDBASE+'/'+'elevationRAW'
LOCATIONSOIL=GISDBASE+'/'+'soilRAW'
LOCATIONLULC=GISDBASE+'/'+'lulcRAW'

<div class="alert alert-block alert-success">
<b>Optional:</b> create directories if directories are previously setup
</div>

In [ ]:
!mkdir {SCRATCH}/{PROJDIR}
!mkdir {RAWGISDIR}
!mkdir {RHESSysDIR}
!mkdir {GISDBASE}
!mkdir {RHESSysDIR}/flows
!mkdir {RHESSysDIR}/worldfiles
!mkdir {RHESSysDIR}/defs
!mkdir {RHESSysDIR}/tecfiles
!mkdir {RBASE}

<div class="alert alert-block alert-success">
<b>Optional:</b>  create GRASS database for the project if it is not previously setup.
</div>

In [ ]:
!grass74 -c {EPSGCODE} -e {LOCATION}

<div class="alert alert-block alert-success">
<b>Optional:</b> install R packages to work with GRASS if packages are not yet installed. (how to know)
</div>

In [ ]:
#-----------------------  make directory to hold scource codes from CRAN and download them
!wget -O {RBASE}/sp_1.3-1.tar.gz https://cran.r-project.org/src/contrib/sp_1.3-1.tar.gz
!wget -O {RBASE}/XML_3.98-1.16.tar.gz https://cran.r-project.org/src/contrib/XML_3.98-1.16.tar.gz
!wget -O {RBASE}/rgdal_1.3-6.tar.gz https://cran.r-project.org/src/contrib/rgdal_1.3-6.tar.gz
!wget -O {RBASE}/rgrass7_0.1-12.tar.gz https://cran.r-project.org/src/contrib/rgrass7_0.1-12.tar.gz
#----------------------- install the downloaded packages into R 3.5.x
!R CMD INSTALL {RBASE}/sp_1.3-1.tar.gz
!R CMD INSTALL {RBASE}/XML_3.98-1.16.tar.gz
!R CMD INSTALL {RBASE}/rgdal_1.3-6.tar.gz
!grass74 {LOCATION}/{MAPSET} --exec R CMD INSTALL {RBASE}/rgrass7_0.1-12.tar.gz

### 1.B upload data

<div class="alert alert-block alert-info">
Upload files to the <project/raw_data>. You may use linux command for upload/download files on UVA Rivanna, which is the host cluster for this Jupyter Notebook.

For example, scp -r <folder/files> [USER]@rivanna.hpc.virginia.edu:/scratch/[USER]/[Project]/raw_data
</div>

<div class="alert alert-block alert-success">
<b>Option 1:</b> Download GIS information from HydroShare
</div>

In [ ]:
HydroShareDownloadZip = '6a304067fba34f0c9890d6295e549bbc.zip'

In [ ]:
from os import listdir
import zipfile

# how to detect already exist unzipped file?
with zipfile.ZipFile(RAWGISDIR+'/'+HydroShareDownloadZip, 'r') as zip_ref:
    zip_ref.extractall(RAWGISDIR)

unzippedpath = RAWGISDIR+'/'+HydroShareDownloadZip.split('.')[0]+"/data/contents/"
onlyfiles = [f for f in listdir(unzippedpath) if f.endswith(".zip")]
with zipfile.ZipFile((unzippedpath+onlyfiles[0]), 'r') as zip_ref:
    zip_ref.extractall(unzippedpath)
    
HydroShareDownloadDIR = unzippedpath+onlyfiles[0].split('.')[0]    
listdir(unzippedpath+onlyfiles[0].split('.')[0])

In [ ]:
HydroShareDownloadDIR = unzippedpath+onlyfiles[0].split('.')[0]    
listdir(HydroShareDownloadDIR)

In [ ]:
# do this mannually now until the problem is solved above.
from os import listdir
HydroShareDownloadDIR = '/scratch/hl8vq/jupyter_ws18/raw_data/6a304067fba34f0c9890d6295e549bbc/data/contents/Coweeta_EJRV'
listdir(HydroShareDownloadDIR)

In [ ]:
downloadedDEMfile = HydroShareDownloadDIR + '/' + 'dem.tif'
#downloadedDEMfile = RAWGISDIR + '/' + "dem.tif"
downloadedLULCfile = HydroShareDownloadDIR + '/' + 'NLCD.tif'
gageShapefile = HydroShareDownloadDIR + '/' + 'coweeta_weirs_shp.shp' # (optional)

<div class="alert alert-block alert-success">
<b>Option 2:</b> Upload files to Rivanna by yourself 
</div>

In [ ]:
downloadedDEMfile = RAWGISDIR + '/' + 'dem.tif'
downloadedLULCfile = RAWGISDIR + '/' + 'NLCD.tif'
downloadedSSURGOdirectory = = RAWGISDIR + '/' + 'MC005'
gageShapefile = RAWGISDIR + '/' + 'usgs.shp' # (optional)

***
## 3) setup GRASS 7.4.x database

### 3.a import elevation from uploaded source

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -e --overwrite input={downloadedDEMfile} output=demRAW location=elevationRAW

<div class="alert alert-block alert-success">
<b>Option 1:</b> re-projection
</div>

In [ ]:
!grass74 {LOCATIONDEM}/{MAPSET} --exec g.region raster=demRAW
!grass74 {LOCATIONDEM}/{MAPSET} --exec r.out.gdal --overwrite input=demRAW output={SCRATCH}/{PROJDIR}/raw_data/dem{RESOLUTION}m.tif format=GTiff
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -o -e --overwrite input={SCRATCH}/{PROJDIR}/raw_data/dem{RESOLUTION}m.tif output=dem
!grass74 {LOCATION}/{MAPSET} --exec g.region raster=dem

<div class="alert alert-block alert-success">
<b>Option 2:</b> re-projection and re-cast spatial resolution
</div>

In [ ]:
!grass74 {LOCATIONDEM}/{MAPSET} --exec g.region raster=demRAW
!grass74 {LOCATIONDEM}/{MAPSET} --exec g.region res={RESOLUTION} -a -p
!grass74 {LOCATIONDEM}/{MAPSET} --exec r.resamp.stats -w input=demRAW output=dem{RESOLUTION}m
!grass74 {LOCATIONDEM}/{MAPSET} --exec r.out.gdal --overwrite input=dem$RESOLUTION'm' output={SCRATCH}/{PROJDIR}/raw_data/dem{RESOLUTION}m.tif format=GTiff
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -o -e --overwrite input={SCRATCH}/{PROJDIR}/raw_data/dem{RESOLUTION}m.tif output=dem
!grass74 {LOCATION}/{MAPSET} --exec g.region raster=dem

### 3.c import soil from uploaded source

<div class="alert alert-block alert-success">
<b>Option 1:</b> import from the downloaded SSURGO
</div>

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec v.in.ogr --overwrite input={downloadedSSURGOdirectory}/spatial/soilmu_a_"$(echo $downloadedSSURGOdirectory | tr '[A-Z]' '[a-z]')".shp output=ssurgo location=soilRAW
!grass74 {LOCATION}/{MAPSET} --exec v.proj --overwrite location=soilRAW mapset=PERMANENT input=ssurgo output=ssurgo
!grass74 {LOCATION}/{MAPSET} --exec v.to.rast --overwrite input=ssurgo use=cat output=soil_ssurgo
!grass74 {LOCATION}/{MAPSET} --exec v.db.select --overwrite map=ssurgo separator=comma file={SCRATCH}/{PROJDIR}/raw_data/soil_cat_mukey.csv
## download R scripts to calculation soil types
!wget -O {RBASE}/ssurgo_extraction.R https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_extraction.R
!wget -O {RBASE}/ssurgo_soiltexture2gis.R https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_soiltexture2gis.R
!Rscript {RBASE}/ssurgo_extraction.R {SCRATCH}/{PROJDIR}/raw_data/{downloadedSSURGOdirectory}
!grass74 {LOCATION}/{MAPSET} --exec Rscript {RBASE}/ssurgo_soiltexture2gis.R {SCRATCH}/{PROJDIR}/raw_data/soil_cat_mukey.csv {SCRATCH}/{PROJDIR}/raw_data/{downloadedSSURGOdirectory}/soil_mukey_texture.csv

<div class="alert alert-block alert-success">
<b>Option 2:</b> manually define by raster calculator
</div>

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec r.mapcalc --overwrite expression="soil_texture = 8"

### 3.d delinearate study catchment, construct drainage structures, and define some RHESSys variables

<div class="alert alert-block alert-info">
<b>Note:</b> We need to define an outlet for a catchment
</div>

<div class="alert alert-block alert-success">
<b>Option 1:</b> use Lat/Long (WSG84)
</div>

In [ ]:
### ... if input is Lat/Long (translate Lat/Long to UTM coordinate and then make a outlet.shp in LOCATION)
%%bash
gageLat='39.47947' # catchment outlet WSG84 Lat (decimal degree)
gageLong='-76.67803' # catchment outlet WSG84 Long (decimal degree; includes the negative sign if applied)
declare $(grass74 $LOCATION/$MAPSET --exec m.proj -i coordinates=$gageLong,$gageLat separator=space | awk '{print "xyCoord=" $1 "," $2}')
echo $xyCoord | grass74 $LOCATION/$MAPSET --exec v.in.ascii in=- out=test x=1 y=2 separator=, --overwrite

<div class="alert alert-block alert-success">
<b>Option 2:</b> upload outlet.shp (we use this option in this example)
</div>

In [ ]:
### ... if input is a shapefile point (import to LOCATIONOUTLET and the reproject to LOCATION as "outlet")
!grass74 {LOCATION}/{MAPSET} --exec v.in.ogr --overwrite input={gageShapefile} output=outlet location=outletRAW
LOCATIONOUTLET = GISDBASE+'/'+"outletRAW"

In [ ]:
!grass74 {LOCATIONOUTLET}/{MAPSET} --exec v.extract --overwrite input=outlet type=point where="COMMENT = 'Weir 18'" output=gage

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec v.proj --overwrite location=outletRAW mapset=PERMANENT input=gage output=outlet

<div class="alert alert-block alert-info">
<b>Tip:</b> Use blue boxes (alert-info) for tips and notes. 
If it’s a note, you don’t have to include the word “Note”.
</div>

In [ ]:
expectedDrainageArea=125700 # meter squre 

GRASS_thres = 1000 # grid cell
GRASS_drainarea_lowerbound = 0.98*expectedDrainageArea/RESOLUTION/RESOLUTION # (allow 2% error)
GRASS_drainarea_upperbound = 1.02*expectedDrainageArea/RESOLUTION/RESOLUTION # (allow 2% error)
!wget -O {RBASE}/grass_delineation.sh https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/grass_delineation.sh
!grass74 {LOCATION}/{MAPSET} --exec bash {RBASE}/grass_delineation.sh {GRASS_thres} {GRASS_drainarea_lowerbound} {GRASS_drainarea_upperbound}



### 3.e define zones

<div class="alert alert-block alert-info">
<b>Note:</b> Zone is for the climate

<div class="alert alert-block alert-success">
<b>Option 1:</b> One zone for the entire catchment
</div>

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec r.mapcalc --overwrite expression="zone = hill"

</div>
<div class="alert alert-block alert-success">
<b>Option 2:</b> define zone by cluster analysis
</div>

In [ ]:
!wget -O {RBASE}/zone_cluster.R https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/zone_cluster.R
!grass74 {LOCATION}/{MAPSET} --exec Rscript {RBASE}/zone_cluster.R dem slope aspect hill

</div>
<div class="alert alert-block alert-success">
<b>Option 3:</b> define zone by patch
</div>

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec r.mapcalc --overwrite expression="zone = patch"

### 3.f import LULC from uploaded scource

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -e --overwrite input={downloadedLULCfile} output=lulcRAW location=lulcRAW
!grass74 {LOCATIONLULC}/{MAPSET} --exec r.out.gdal --overwrite input=lulcRAW output={SCRATCH}/{PROJDIR}/raw_data/LULC{RESOLUTION}m.tif format=GTiff
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -o --overwrite input={SCRATCH}/{PROJDIR}/raw_data/LULC{RESOLUTION}m.tif output=NLCD

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec r.stats input=NLCD -c

<div class="alert alert-block alert-success">
<b>Optional:</b> define landuse/vegetation/lai/imprevious/cover fraction from NLCD code
</div>

In [ ]:
!wget -O {RBASE}/NLCD2RHESSys.R https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/NLCD2RHESSys.R
!grass74 {LOCATION}/{MAPSET} --exec Rscript {RBASE}/NLCD2RHESSys.R

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec g.list -p type='rast' | cat

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec r.stats input=impervious -c

### 3.h additional customizations

define roads

<div class="alert alert-block alert-success">
<b>Option 1:</b> upload road.shp and the rasterize it
</div>

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec v.in.ogr --overwrite input={SCRATCH}/{PROJDIR}/raw_data/{downloadedROADfile} output=roads location=roadRAW
!grass74 {LOCATION}/{MAPSET} --exec v.proj --overwrite location=roadRAW mapset=PERMANENT input=roads output=roads
!grass74 {LOCATION}/{MAPSET} --exec v.to.rast --overwrite input=roads@PERMANENT output=roads use=cat

<div class="alert alert-block alert-success">
<b>Option 2:</b> upload road raster
</div>

In [ ]:
downloadedROADfile = HydroShareDownloadDIR + '/' + 'roads.tif'
LOCATIONROAD = GISDBASE+'/'+'roadRAW'
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -e --overwrite input={downloadedROADfile} output=roadRAW location=roadRAW
!grass74 {LOCATIONROAD}/{MAPSET} --exec r.out.gdal --overwrite input=roadRAW output={SCRATCH}/{PROJDIR}/raw_data/ROAD{RESOLUTION}m.tif format=GTiff
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -o --overwrite input={SCRATCH}/{PROJDIR}/raw_data/ROAD{RESOLUTION}m.tif output=roads

define isohyet 

<div class="alert alert-block alert-success">
<b>Option 1:</b> no isohyet
</div>

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec r.mapcalc --overwrite expression="isohyet = 1"

<div class="alert alert-block alert-success">
<b>Option 2:</b> upload road raster
</div>

In [ ]:
downloadedISOHYETfile = HydroShareDownloadDIR + '/' + 'isohyet.tif'
LOCATIONISOHYET = GISDBASE+'/'+'isohyetRAW'
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -e --overwrite input={downloadedISOHYETfile} output=isohyetRAW location=isohyetRAW
!grass74 {LOCATIONISOHYET}/{MAPSET} --exec r.out.gdal --overwrite input=roadRAW output={SCRATCH}/{PROJDIR}/raw_data/ISOHYET{RESOLUTION}m.tif format=GTiff
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -o --overwrite input={SCRATCH}/{PROJDIR}/raw_data/ISOHYET{RESOLUTION}m.tif output=isohyet

***
## 4) constructing worldfile and flowtable to RHESSys

<div class="alert alert-block alert-info">
<b>Note:</b> This is the most important part

<div class="alert alert-block alert-success">
<b>Option1:</b> copy climate series data from HydroShare download
</div>

In [ ]:
from os import listdir 
#!cp -r {HydroShareDownloadDIR}/rhessys/clim {SCRATCH}/{PROJDIR}/{RHESSysMODEL}
onlyfiles = [f for f in listdir(SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL+'/clim') if f.endswith(".base")]
tmp = !head -n1 {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/clim/{onlyfiles[0]}
climateBaseFile = 'clim/'+onlyfiles[0]
climateBaseID = tmp.fields(0)[0]

#!wget -O {RBASE}/g2w.R https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/g2w.R
#!wget -O {RBASE}/vegCollection.csv https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/vegCollection.csv
#!wget -O {RBASE}/soilCollection.csv https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/soilCollection.csv
#!wget -O {RBASE}/lulcCollection.csv https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulcCollection.csv
!grass74 {LOCATION}/{MAPSET} --exec Rscript {RBASE}/g2w.R {SCRATCH}/{PROJDIR} {climateBaseID} {climateBaseFile} {RBASE}/vegCollection.csv {RBASE}/soilCollection.csv {RBASE}/lulcCollection.csv {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/worldfiles/worldfile.csv {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/worldfiles/worldfile.hdr {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/defs

<div class="alert alert-block alert-success">
<b>Option2:</b> define climate info by user
</div>

In [ ]:
climateBaseFile = 'clim/SLB.base'
climateBaseID = 101    

!wget -O {RBASE}/g2w.R https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/g2w.R
!wget -O {RBASE}/vegCollection.csv https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/1mUrban/vegCollection.csv
!wget -O {RBASE}/soilCollection.csv https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/1mUrban/soilCollection.csv
!wget -O {RBASE}/lulcCollection.csv https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/1mUrban/lulcCollection.csv
!grass74 {LOCATION}/{MAPSET} --exec Rscript {RBASE}/g2w.R {SCRATCH}/{PROJDIR} {climateBaseID} {climateBaseFile} {RBASE}/vegCollection.csv {RBASE}/soilCollection.csv {RBASE}/lulcCollection.csv {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/worldfiles/worldfile.csv {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/worldfiles/worldfile.hdr {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/defs

In [ ]:
!wget -O {RBASE}/LIB_RHESSys_writeTable2World.R https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/LIB_RHESSys_writeTable2World.R
!Rscript {RBASE}/LIB_RHESSys_writeTable2World.R na {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/worldfiles/worldfile.csv {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/worldfiles/worldfile


In [ ]:
!wget -O {RBASE}/createFlowRouting.R https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/createFlowRouting.R
!grass74 {LOCATION}/{MAPSET} --exec Rscript {RBASE}/createFlowRouting.R {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/flows/flowtable.txt



****
***
***
# testing section

In [ ]:
pwd

In [ ]:
!grass74 {LOCATIONDEM}/{MAPSET} --exec Rscript install.R

In [ ]:
print(RBASE)

In [ ]:
!echo {LOCATIONDEM}

In [ ]:
!grass74 /scratch/hl8vq/jupyter_ws18/grass_dataset/outletRAW/{MAPSET} --exec v.to.db map=outlet type=point -p option=coor


In [ ]:
!grass74 /scratch/hl8vq/jupyter_ws18/grass_dataset/outletRAW/{MAPSET} --exec db.describe -c outlet



# comments below

In [ ]:
%env

In [ ]:
%R
library(sp,lib='~/rlib')

In [ ]:
!grass74 --version

In [ ]:
!R --version

In [ ]:
!Rscript

In [ ]:
PROJDIR = 'test1'
!mkdir {PROJDIR}
!mkdir {PROJDIR}/raw_data

!ls -l {PROJDIR}

In [ ]:
contents = !ls
print(contents)

In [ ]:
%load https://raw.githubusercontent.com/dib-lab/khmer/master/scripts/fastq-to-fasta.py
!curl -O https://raw.githubusercontent.com/dib-lab/khmer/master/scripts/fastq-to-fasta.py


In [ ]:
m = Map(center=[35.049120, -83.433894], zoom=15)
with open('coweeta_18.geojson') as f:
    data = json.load(f)
g = GeoJSON(data=data)
m.add_layer(g)
m

## 2) Description of Sensitivity parameter

### 1) -s

-s : value1, value2, value3 
  * the m, K, and soil depth parameter value initialized for each patch in the worldfile are multiplied by 1) value1 2) value2 and 3) value3 respectively during a simulation.)

 - value1 : m (the decay of hydraulic conductivity with depth)
 
 - value2 : K (hydraulic conductivity at the surface)
 
 - value3 : soil depth (hydraulic conductivity at the surface)

### 2) -sv

-sv : value1, value2 
* (the m, K are multipliers to scale the vertical decay of hydraulic conductivity with depth (m), and vertical hydraulic conductivity at the surface (K).

### 3) -gw

-gw : value1, value2 
 * value1 : The first value is a multiplier on the sat_to_gw_coeff parameter set in the soil definition file (representing the amount of water moving from the saturated store to the groundwater store).
  - sat_to_gw_coeff(%) : the amount of water moving from the saturated store to the groundwater store; bypasses roots
 * value2 : The second value is a multiplier on the gw_loss_coeff parameter in the hillslope default file (representing the amount of water moving from the groundwater store to the stream).
  - gw_loss_coeff(%) : Percent of groundwater store lost to stream

### 4) Parameter & typical range : m (0.01~20), Ksat0 (1~600), gw1 (0.001~0.3), gw2 (0.01~0.9)

## 3) Set sensitivity parameters of RHESSys Model

In [ ]:
executable_file = "./RHESSys5.20.source/rhessys/rhessys5.20.0"

In [ ]:
start_date = '2000 1 1 1'

In [ ]:
end_date = '2008 10 1 1'

In [ ]:
# -b only basin output; -gwtoriparian receiving groundwater and put in stream
unknown_cmd = "-b -newcaprise -capr 0.001 -gwtoriparian -capMax 0.01 -slowDrain -leafDarkRespScalar 0.5 -frootRespScalar 0.25 -StemWoodRespScalar 0.05 -t tecfiles/tec_daily.txt -w worldfiles/world -whdr worldfiles/world.hdr -r flows/flow.txt -rtz 2.7"

In [ ]:
ratio = 0.1 # 0, -0.1

In [ ]:
s_value1 = str(2.9 + 2.9*ratio)
s_value2 = str(1.4 + 1.4*ratio)
s_value3 = str(20.0 + 20.0*ratio)

In [ ]:
sv_value1 = str(4.5 + 4.5*ratio)  
sv_value2 = str(55.6 + 55.6*ratio)  

In [ ]:
gw_value1 = str(0.05)   
gw_value2 = str(0.1)  #calibrated values  

In [ ]:
output_prefix = "rhessys08"

In [ ]:
cmd = "{} -st {} -ed {} {} -pre output/{} -s {} {} {} -sv {} {} -gw {} {}".format(executable_file, start_date, 
                                                                                  end_date, unknown_cmd, output_prefix, 
                                                                                  s_value1, s_value2, s_value3, 
                                                                                  sv_value1, sv_value2, 
                                                                                  gw_value1, gw_value2)

## 4) run RHESSys Model

In [ ]:
import subprocess
import shlex

In [ ]:
cmd = shlex.split(cmd)
p = subprocess.Popen(cmd, stdout=subprocess.PIPE)
output = p.communicate()[0].decode('utf-8')
print(output)

In [ ]:
cmd = shlex.split(cmd)
p = subprocess.Popen(cmd, stdout=subprocess.PIPE)

In [ ]:
p.communicate()[0].decode('utf-8')

In [ ]:
!./RHESSys5.20.source/rhessys/rhessys5.20.0 -st 2000 1 1 1 -ed 2003 10 1 1 \
  -b -newcaprise -capr 0.001 -gwtoriparian -capMax 0.01 -slowDrain -leafDarkRespScalar 0.5 \
  -frootRespScalar 0.25 -StemWoodRespScalar 0.05 \
  -t tecfiles/tec_daily.txt -w worldfiles/world -whdr worldfiles/world.hdr -r flows/flow.txt -rtz 2.7 \
  -pre output/rhessys_m6K3 -s 6 3 20.0 -sv 4.5 55.6 -gw 0.05 0.1

## 5) Plotting of RHESSys Model Output

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

 ## Basin Daily Output
 |                    RHESSys Output Abbreviation                   | Description |   Units |   
 |---------------------------------------------|-------------|-------------------|
 |         pot_surface_infil| Rain Throughfall    | mm          | 
 |       snow_thr        | Snow Throughfall    | mm         | 
 |sat_def_z  | Saturation Deficit with depth    | mm of depth          | 
 |sat_def | Saturation Deficit - volume  | mm of water       | 
 |rz_storage	|Rooting Zone Storage	|mm of water
 |unsat_stor|	Unsaturated Storage	|mm
 |rz_drainage|	Rooting Zone Drainage|	mm
 |unsat_drain|	Unsaturated| Storage	mm
 |cap	|Capillary Rise|	mm
 |evap	|Evaporation|	mm
 |snowpack	|Snow Water Equivalent (SWE)|	mm
 |trans	|Transpiration|	mm
 |baseflow	|Baseflow	|mm
 |return	|Return flow|	mm
 |streamflow	|Total Stream Outflow|	mm (normalized by basin area)
 | psn	|Net Photosynthesis	|kgC/m2
|lai	|Leaf Area Index	|m2/m2
|gw.Qout	|Groundwater Output	|mm
|gw.storage	|Groundwater Store	|mm
|detention_store|	Detention Store	|mm
|%sat_area|	Percent Saturated Area	|m2/m2
|litter_store|	Litter intercepted water Store	|m2/m2
|canopy_store|	Canopy Intercepted water Store	|m2/m2
|%snow_cover|	Percent Snow Cover	|m2/m2
|snow_subl|	Snow Sublimation	|
|trans_var|	Spatial variation in transpiration	|
|acc_trans		||
|acctransv_var		||
|pet	|Potential Evapotranspiration|	mm
|dC13		||
|precip	|Precipitation|	mm
|pcp_assim||		
|mortf	|Fraction of Basin that have tree mortality	|
|tmax	|Maximum Temperature	|°C
|tmin	|Minimum Temperature	|°C
|tavg	|Average Temperature	|°C
|vpd	|Vapor Pressure Deficit	|Pa
|snowfall	|Snowfall	|
|recharge	|_Recharge of water to soil	|
|gpsn	|Gross Photosynthesis	|kgC/m2
|resp	|_ Respiration_	|kgC/m2
|gs	|Canopy Conductance	|mm/s?
|rootdepth	|Rooting depth	|
|plantc	|Plant Carbon	|kgC/m2
|snowmelt	|Snow Melt	|
|canopysubl	|Canopy Sublimation	|
|routedstreamflow	||	
|canopy_snow	|Snow Intercepted on Canopy	|
|height	|Canopy height	|
|evap_can	|Canopy Evaporation?	|
|evap_lit	|Litter Evaporation_	|
|evap_soil	|Soil Evaporation_	|
|litrc	|Litter Carbon_	|
|Kdown	|Downward (from atmosphere) Direct Shortwave Radiation_	|
|Ldown	|Downward (from atmosphere) Longwave Radiation_	|
|Kup	|Reflected (upward) Shortwave Radiation_	|
|Lup	|Reflected (upward) Longwave Radiation_	|
|Kstar_can	|Absorbed shortwave by canopy	|
|Kstar_soil	|Absorbed shortwave by soil	|
|Kstar_snow	|Absorbed shortwave bysnow	|
|Lstar_can	|Absorbed longwave by canopy	|
|Lstar_soil	|Absorbed longwave by soil	|
|Lstar_snow	|Absorbed longwave by snow	|
|LE_canopy	|Latent heat evaporated by canopy	|
|LE_soil	La	||
|LE_snow		||
|Lstar_strat		||
|canopydrip		||
|ga	|Aerodynamic Conductance	|mm/s

In [ ]:
path = "output/"
#basin_daily_output = pd.read_csv(path + output_prefix +"_basin.daily", delimiter=" ")
basin_daily_output = pd.read_csv(path + "rhessys_m6K3_basin.daily", delimiter=" ")

In [ ]:
start_date = "2000-01-01"
end_date = "2001-09-30"
#end_date = "2008-09-30"

In [ ]:
date_index = pd.date_range(start_date, end_date, freq='1D')
basin_daily_output_date = basin_daily_output.insert(loc=0, column='Date', value=date_index.values)
#basin_daily_output_date1 = basin_daily_output.insert1(loc=0, column='Date', value=date_index.values)
basin_daily_output_date_index = basin_daily_output.set_index('Date')
#basin_daily_output_date1_index = basin_daily_output1.set_index('Date')

In [ ]:
basin_daily_output_date_index.head()

In [ ]:
plt_start_date = '2001-01-01'
plt_end_date = '2001-09-30'

In [ ]:
basin_daily_output_f = basin_daily_output_date_index.loc[plt_start_date:plt_end_date]
basin_daily_output1_f = basin_daily_output_date1_index.loc[plt_start_date:plt_end_date]

In [ ]:
# Rain Throughfall (mm)
basin_daily_output_f['pot_surface_infil'].plot(figsize=(17,5))

In [ ]:
# Saturation Deficit with depth (mm of depth)
basin_daily_output_f['sat_def_z'].plot(figsize=(17,5))
basin_daily_output1_f['sat_def_z'].plot(figsize=(17,5))

In [ ]:
# Saturation Deficit - volume (mm of water)
basin_daily_output_f['sat_def'].plot(figsize=(17,5))

In [ ]:
# Rooting Zone Storage  (mm of water)
basin_daily_output_f['rz_storage'].plot(figsize=(17,5))

In [ ]:
# Unsaturated Storage (mm)
basin_daily_output_f['unsat_stor'].plot(figsize=(17,5))

In [ ]:
# Rooting Zone Drainage (mm)
basin_daily_output_f['rz_drainage'].plot(figsize=(17,5))

In [ ]:
# Unsaturated Drainage (mm)
basin_daily_output_f['unsat_drain'].plot(figsize=(17,5))

In [ ]:
# Capillary Rise (mm)
basin_daily_output_f['cap'].plot(figsize=(17,5))

In [ ]:
# Evaporation (mm)
basin_daily_output_f['evap'].plot(figsize=(17,5))

In [ ]:
# Snow Water Equivalent (SWE) (mm)
basin_daily_output_f['snowpack'].plot(figsize=(17,5))

In [ ]:
# Transpiration (mm)
basin_daily_output_f['trans'].plot(figsize=(17,5))

In [ ]:
# Baseflow (mm)
basin_daily_output_f['baseflow'].plot(figsize=(17,5))

In [ ]:
# Return flow (mm)
basin_daily_output_f['return'].plot(figsize=(17,5))

In [ ]:
# Total Stream Outflow (mm (normalized by basin area))
basin_daily_output_f['streamflow'].plot(figsize=(17,5))

In [ ]:
# Net Photosynthesis (kgC/m2)
basin_daily_output_f['psn'].plot(figsize=(17,5))

In [ ]:
# Leaf Area Index (m2/m2)
basin_daily_output_f['lai'].plot(figsize=(17,5))

In [ ]:
# Groundwater Output (mm)
basin_daily_output_f['gw.Qout'].plot(figsize=(17,5))

In [ ]:
# Groundwater Store (mm)
basin_daily_output_f['gw.storage'].plot(figsize=(17,5))

In [ ]:
# Detention Store (mm)
basin_daily_output_f['detention_store'].plot(figsize=(17,5))

In [ ]:
# Percent Saturated Area (m2/m2)
basin_daily_output_f['%sat_area'].plot(figsize=(17,5))

In [ ]:
# Litter intercepted water Store (m2/m2)
basin_daily_output_f['litter_store'].plot(figsize=(17,5))

In [ ]:
# Canopy Intercepted water Store (m2/m2)
basin_daily_output_f['canopy_store'].plot(figsize=(17,5))

In [ ]:
# Percent Snow Cover (m2/m2)
basin_daily_output_f['%snow_cover'].plot(figsize=(17,5))

In [ ]:
# Snow Sublimation
basin_daily_output_f['snow_subl'].plot(figsize=(17,5))

In [ ]:
# Spatial variation in transpiration
basin_daily_output_f['trans_var'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['acc_trans'].plot(figsize=(17,5))

In [ ]:
# Potential Evapotranspiration
basin_daily_output_f['pet'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['dC13'].plot(figsize=(17,5))

In [ ]:
# Precipitation (mm)
basin_daily_output_f['precip'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['pcp_assim'].plot(figsize=(17,5))

In [ ]:
# Fraction of Basin that have tree mortality
basin_daily_output_f['mortf'].plot(figsize=(17,5))

In [ ]:
# Maximum Temperature (°C)
basin_daily_output_f['tmax'].plot(figsize=(17,5))

In [ ]:
# Minimum Temperature (°C)
basin_daily_output_f['tmin'].plot(figsize=(17,5))

In [ ]:
# Average Temperature (°C)
basin_daily_output_f['tavg'].plot(figsize=(17,5))

In [ ]:
# Vapor Pressure Deficit (°C)
basin_daily_output_f['vpd'].plot(figsize=(17,5))

In [ ]:
# Snowfall
basin_daily_output_f['snowfall'].plot(figsize=(17,5))

In [ ]:
# _Recharge of water to soil
basin_daily_output_f['recharge'].plot(figsize=(17,5))

In [ ]:
# _Gross Photosynthesis (kgC/m2)
basin_daily_output_f['gpsn'].plot(figsize=(17,5))

In [ ]:
# _ Respiration_ (kgC/m2)
basin_daily_output_f['resp'].plot(figsize=(17,5))

In [ ]:
# Canopy Conductance (mm/s?)
basin_daily_output_f['gs'].plot(figsize=(17,5))

In [ ]:
# Rooting depth
basin_daily_output_f['rootdepth'].plot(figsize=(17,5))

In [ ]:
# Plant Carbon (kgC/m2)
basin_daily_output_f['plantc'].plot(figsize=(17,5))

In [ ]:
# Snow Melt
basin_daily_output_f['snowmelt'].plot(figsize=(17,5))

In [ ]:
# Canopy Sublimation
basin_daily_output_f['canopysubl'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['routedstreamflow'].plot(figsize=(17,5))

In [ ]:
# Snow Intercepted on Canopy
basin_daily_output_f['canopy_snow'].plot(figsize=(17,5))

In [ ]:
# Canopy height
basin_daily_output_f['height'].plot(figsize=(17,5))

In [ ]:
# Canopy Evaporation?
basin_daily_output_f['evap_can'].plot(figsize=(17,5))

In [ ]:
# Litter Evaporation_
basin_daily_output_f['evap_lit'].plot(figsize=(17,5))

In [ ]:
# Soil Evaporation_
basin_daily_output_f['evap_soil'].plot(figsize=(17,5))

In [ ]:
# Litter Carbon_
basin_daily_output_f['litrc'].plot(figsize=(17,5))

In [ ]:
# Downward (from atmosphere) Direct Shortwave Radiation_
basin_daily_output_f['Kdown'].plot(figsize=(17,5))

In [ ]:
# Downward (from atmosphere) Longwave Radiation_
basin_daily_output_f['Ldown'].plot(figsize=(17,5))

In [ ]:
# Reflected (upward) Shortwave Radiation_
basin_daily_output_f['Kup'].plot(figsize=(17,5))

In [ ]:
# Reflected (upward) Longwave Radiation_
basin_daily_output_f['Lup'].plot(figsize=(17,5))

In [ ]:
# Absorbed shortwave by canopy
basin_daily_output_f['Kstar_can'].plot(figsize=(17,5))

In [ ]:
# Absorbed shortwave by soil
basin_daily_output_f['Kstar_soil'].plot(figsize=(17,5))

In [ ]:
# Absorbed shortwave bysnow
basin_daily_output_f['Kstar_snow'].plot(figsize=(17,5))

In [ ]:
# Absorbed longwave by canopy
basin_daily_output_f['Lstar_can'].plot(figsize=(17,5))

In [ ]:
# Absorbed longwave by soil
basin_daily_output_f['Lstar_soil'].plot(figsize=(17,5))

In [ ]:
# Absorbed longwave by snow
basin_daily_output_f['Lstar_snow'].plot(figsize=(17,5))

In [ ]:
# Latent heat evaporated by canopy
basin_daily_output_f['LE_canopy'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['LE_soil'].plot(figsize=(17,5))

In [ ]:
# LE_snow
basin_daily_output_f['LE_snow'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['Lstar_strat'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['canopydrip'].plot(figsize=(17,5))

In [ ]:
# Aerodynamic Conductance
basin_daily_output_f['ga'].plot(figsize=(17,5))

In [ ]:
plt.figure(1)
plt.subplot(2,3,1)

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax = fig.add_subplot(331)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['precip'])
plt.legend()
ax = fig.add_subplot(332)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['pet'])
plt.legend()
ax = fig.add_subplot(333)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['evap'])
plt.legend()
ax = fig.add_subplot(334)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['evap_can'])
plt.legend()
ax = fig.add_subplot(335)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['evap_lit'])
plt.legend()
ax = fig.add_subplot(336)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['evap_soil'])
plt.legend()
ax = fig.add_subplot(337)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['trans'])
plt.legend()
ax = fig.add_subplot(338)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['psn'])
plt.legend()
ax = fig.add_subplot(339)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['lai'])
plt.legend()

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax = fig.add_subplot(331)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['precip'])
plt.legend()
ax = fig.add_subplot(332)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['streamflow'])
plt.legend()
ax = fig.add_subplot(333)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['return'])
plt.legend()
ax = fig.add_subplot(334)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['baseflow'])
plt.legend()
ax = fig.add_subplot(335)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['unsat_drain'])
plt.legend()
ax = fig.add_subplot(336)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['rz_drainage'])
plt.legend()
ax = fig.add_subplot(337)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['gw.Qout'])
plt.legend()
ax = fig.add_subplot(338)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['%sat_area'])
plt.legend()
ax = fig.add_subplot(339)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['tavg'])
plt.legend()

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax = fig.add_subplot(331)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['sat_def_z'])
plt.legend()
ax = fig.add_subplot(332)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['sat_def'])
plt.legend()
ax = fig.add_subplot(333)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['cap'])
plt.legend()
ax = fig.add_subplot(334)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['gw.storage'])
plt.legend()
ax = fig.add_subplot(335)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['detention_store'])
plt.legend()
ax = fig.add_subplot(336)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['litter_store'])
plt.legend()
ax = fig.add_subplot(337)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['canopy_store'])
plt.legend()
ax = fig.add_subplot(338)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['vpd'])
plt.legend()
ax = fig.add_subplot(339)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['recharge'])
plt.legend()

## 6) Validation between the observation and simulation data.

In [ ]:
path = "obs/"
obs_streamflow = pd.read_csv(path + "Qobs_18_r.csv") #, header=3
obs_streamflow.head()

In [ ]:
start_date = "1936-11-01"
date_index1 = pd.date_range(start_date, periods=len(obs_streamflow), freq='1D')
obs_streamflow_date = obs_streamflow.insert(loc=0, column='Date', value=date_index1.values)
obs_streamflow_date_index = obs_streamflow.set_index('Date')
obs_streamflow_date_index.head()

In [ ]:
obs_streamflow_filt = obs_streamflow_date_index.loc[plt_start_date:plt_end_date]

In [ ]:
# create the plot figure 
plt.figure(figsize=(15,5))
# get the current axis of the plot
ax = plt.gca()
# plot and set label, marker, and markersize
ax.plot(obs_streamflow_filt['discharge (mm)'], label='Observation(mm)', marker="^", markersize=3)
ax.plot(basin_daily_output_f['streamflow'], label='Model Output(mm)', marker="*", markersize=3)
ax.grid(True)
# set the y-axis labels
ax.set_ylabel('Streaflow(m)', fontsize=15)
# setting legend, xticks and yticks fontsizes
plt.legend(fontsize=12)
plt.xticks(rotation=90, fontsize=12)
plt.yticks(fontsize=12)
plt.show()

#### Application of validation method

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
from pysumma.Validation import validation

In [ ]:
# defind simulation & observation data
lumped_simulation_streamflow = basin_daily_output_f['streamflow'].fillna(0)
observation_streamflow = obs_streamflow_filt['discharge (mm)'].fillna(0)

In [ ]:
# analyze validtation between 1d richards' runoff simulation and observation data.
validation.analysis(observation_streamflow, lumped_simulation_streamflow)

In [ ]:
r2_score(observation_streamflow, lumped_simulation_streamflow)

In [ ]:
#bias NSE log.Q
#monthly, daily, weekly